In [23]:
import warnings
warnings.filterwarnings(action='ignore')

import pandas as pd
import numpy as np 
pd.set_option('float_format', '{:f}'.format)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt 

plt.style.use('seaborn')
plt.rc('font', family='AppleGothic')
plt.rc('axes', unicode_minus=False)

import requests
from tqdm import tqdm, tqdm_notebook

from tslearn.clustering import TimeSeriesKMeans

In [2]:
# 온라인 구매 이력
# 날짜, 성별, 연령대, 큰 카테고리, 작은 카테고리, 구매량
buy2018_1 = pd.read_csv('buy2018_1.csv', encoding='cp949')
buy2018_2 = pd.read_csv('buy2018_2.csv', encoding='cp949') 
buy2019_1 = pd.read_csv('buy2019_1.csv', encoding='cp949') 
buy2019_2 = pd.read_csv('buy2019_2.csv', encoding='cp949')

buy2018_1 = buy2018_1.iloc[:,1:]
buy2018_2 = buy2018_2.iloc[:,1:]
buy2019_1 = buy2019_1.iloc[:,1:]
buy2019_2 = buy2019_2.iloc[:,1:]

buy2018_1.columns = ['date', 'sex', 'age', 'big_cat', 'sm_cat', 'qty']
buy2018_2.columns = ['date', 'sex', 'age', 'big_cat', 'sm_cat', 'qty']
buy2019_1.columns = ['date', 'sex', 'age', 'big_cat', 'sm_cat', 'qty']
buy2019_2.columns = ['date', 'sex', 'age', 'big_cat', 'sm_cat', 'qty']


# 소셜 데이터 
# 날짜, 큰 카테고리, 작은 카테고리, 구매량
sns2018_1 = pd.read_csv('sns2018_1.csv', encoding='cp949') 
sns2018_2 = pd.read_csv('sns2018_2.csv', encoding='cp949') 
sns2019_1 = pd.read_csv('sns2019_1.csv', encoding='cp949') 
sns2019_2 = pd.read_csv('sns2019_2.csv', encoding='cp949')

sns2018_1 = sns2018_1.iloc[:,1:]
sns2018_2 = sns2018_2.iloc[:,1:]
sns2019_1 = sns2019_1.iloc[:,1:]
sns2019_2 = sns2019_2.iloc[:,1:]

sns2018_1.columns = ['date', 'big_cat', 'sm_cat', 'cnt']
sns2018_2.columns = ['date', 'big_cat', 'sm_cat', 'cnt']
sns2019_1.columns = ['date', 'big_cat', 'sm_cat', 'cnt']
sns2019_2.columns = ['date', 'big_cat', 'sm_cat', 'cnt']

In [3]:
buy = pd.concat([buy2018_1, buy2018_2, buy2019_1, buy2019_2], axis=0, ignore_index = True)
sns = pd.concat([sns2018_1, sns2018_2, sns2019_1, sns2019_2], axis=0, ignore_index = True)

df = buy.merge(sns, how='left', on=['date', 'big_cat', 'sm_cat'])

print(df.shape)
df.head()

(2056899, 7)


,date,sex,age,big_cat,sm_cat,qty,cnt
0,20180101,F,20,식품,가공란,37,0.480964
1,20180101,F,30,식품,가공란,16,0.480964
2,20180101,F,40,식품,가공란,9,0.480964
3,20180101,F,50,식품,가공란,3,0.480964
4,20180101,M,20,식품,가공란,13,0.480964


In [4]:
tqdm.pandas()
df["date"] = df['date'].progress_apply(lambda x: pd.to_datetime(str(x), format='%Y-%m-%d'))

100%|██████████| 2056899/2056899 [02:36<00:00, 13153.60it/s]


In [5]:
df_cat = df.groupby(['date', 'big_cat', 'sm_cat']).sum('qty').reset_index()[["date", "big_cat", "sm_cat", "qty"]]

In [6]:
df_appliance = df_cat[df_cat.big_cat == '냉난방가전'].reset_index(drop=True).drop('big_cat', axis=1)
df_beauty = df_cat[df_cat.big_cat == '뷰티'].reset_index(drop=True).drop('big_cat', axis=1)
df_food = df_cat[df_cat.big_cat == '식품'].reset_index(drop=True).drop('big_cat', axis=1)

df_appliance.shape, df_beauty.shape, df_food.shape

((23432, 3), (91562, 3), (152713, 3))

In [7]:
df_appliance.head()

,date,sm_cat,qty
0,2018-01-01,가열식 가습기,7
1,2018-01-01,공기정화 용품,120
2,2018-01-01,공기청정기,106
3,2018-01-01,냉풍기,1
4,2018-01-01,돈풍기,2


In [8]:
df_app_ts = pd.pivot_table(df_appliance, index="sm_cat", columns='date', values='qty')
df_bt_ts = pd.pivot_table(df_beauty, index="sm_cat", columns='date', values='qty')
df_food_ts = pd.pivot_table(df_food, index="sm_cat", columns='date', values='qty')

df_app_ts.shape, df_bt_ts.shape, df_food_ts.shape

((40, 730), (131, 730), (212, 730))

In [9]:
df_app_ts.head()

date,2018-01-01,2018-01-02,2018-01-03,2018-01-04,2018-01-05,2018-01-06,2018-01-07,2018-01-08,2018-01-09,2018-01-10,...,2019-12-22,2019-12-23,2019-12-24,2019-12-25,2019-12-26,2019-12-27,2019-12-28,2019-12-29,2019-12-30,2019-12-31
sm_cat,,,,,,,,,,,,,,,,,,,,,
가스온수기,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,1.000000,nan,nan,nan,nan
가열식 가습기,7.000000,23.000000,12.000000,15.000000,18.000000,11.000000,17.000000,15.000000,16.000000,16.000000,...,27.000000,29.000000,19.000000,10.000000,20.000000,19.000000,14.000000,7.000000,18.000000,15.000000
공기정화 용품,120.000000,191.000000,161.000000,118.000000,196.000000,98.000000,100.000000,204.000000,197.000000,110.000000,...,155.000000,200.000000,245.000000,182.000000,171.000000,148.000000,136.000000,105.000000,196.000000,211.000000
공기청정기,106.000000,147.000000,103.000000,77.000000,141.000000,74.000000,62.000000,166.000000,90.000000,96.000000,...,64.000000,112.000000,124.000000,102.000000,484.000000,95.000000,55.000000,85.000000,135.000000,78.000000
난방용 열풍기,nan,nan,nan,1.000000,1.000000,1.000000,nan,1.000000,2.000000,1.000000,...,nan,1.000000,1.000000,nan,nan,1.000000,nan,nan,1.000000,1.000000


In [78]:
# 가전 (40)
df_app_ts.isnull().sum(axis=1).sort_values(ascending=False) / 730

sm_cat
산림욕기         0.871233
중대형 에어컨      0.853425
가스온수기        0.832877
난방용 열풍기      0.821918
에어커튼         0.690411
천장형 에어컨      0.531507
돈풍기          0.364384
벽걸이형 냉온풍기    0.342466
컨벡터          0.294521
신발건조기        0.267123
가열식 가습기      0.234247
업소용 선풍기      0.223288
멀티형 에어컨      0.182192
보일러          0.156164
냉풍기          0.146575
카페트매트        0.135616
자연식 가습기      0.126027
라디에이터        0.126027
황토매트         0.126027
에어컨 리모컨      0.124658
이동형 에어컨      0.108219
벽걸이형 선풍기     0.093151
탁상/USB 선풍기   0.052055
에어워셔         0.043836
온풍기          0.043836
휴대용 선풍기      0.026027
벽걸이 에어컨      0.026027
전기온수기        0.026027
스탠드형 냉온풍기    0.008219
온수매트         0.008219
스탠드형 에어컨     0.004110
복합식 가습기      0.004110
전기장판         0.004110
초음파식 가습기     0.002740
온열매트         0.000000
의류건조기        0.000000
제습기          0.000000
공기청정기        0.000000
공기정화 용품      0.000000
히터           0.000000
dtype: float64

In [79]:
# 뷰티 (131)
df_bt_ts.isnull().sum(axis=1).sort_values(ascending=False) / 730

sm_cat
여성향수                0.984932
색조 메이크업 세트          0.908219
남성향수                0.901370
기능성 화이트닝 화장품        0.806849
향수세트                0.701370
선파우더                0.384932
헤어무스                0.204110
기능성 링클케어 화장품        0.176712
태닝용 선크림             0.097260
선스프레이               0.080822
팩도구                 0.065753
풋스프레이               0.060274
네일아트                0.054795
바디케어용 땀패드           0.052055
베이스 메이크업 세트         0.016438
색조 메이크업 립라이너        0.015068
뷰티용 샤프너             0.013699
메이크업 박스             0.008219
남성 선케어              0.008219
기능성 화장품 세트          0.008219
스킨케어 곡물팩            0.006849
바디케어용 슬리밍           0.005479
뷰티 타투               0.004110
남성 크림               0.002740
샤워코롱                0.001370
헤어젤                 0.001370
베이스 메이크업용 CC크림      0.001370
베이스 메이크업용 BB크림      0.000000
베이스 메이크업용 메이크업베이스   0.000000
베이스 메이크업용 파운데이션     0.000000
미용가위                0.000000
바디 보습제              0.000000
베이스 메이크업용 컨실러       0.000000
바디 세트               0.000000
바디 스크럽 

In [82]:
# 푸드 (212)
df_food_ts.isnull().sum(axis=1).sort_values(ascending=False) / 730

sm_cat
과일채소 분말/분태      0.647945
마/야콘            0.564384
영지버섯            0.326027
구이/수육용 돈육       0.280822
상황버섯            0.263014
혼합곡             0.204110
커피용 프림          0.094521
건어물 건새우         0.076712
우엉/연근           0.075342
갈비용 우육          0.072603
옥돔              0.064384
홍어              0.031507
미나리             0.026027
굴 생물            0.015068
헛개/가시오가피        0.009589
과일세트            0.006849
백김치             0.006849
감말랭이            0.004110
초유 영양제          0.004110
카페용 초콜릿시럽       0.004110
견과류 캐슈넛         0.004110
절임배추/김치속        0.002740
둥굴레차            0.002740
믹스 채소           0.002740
견과류 피스타치오       0.002740
곶감/반건시          0.001370
장조림/카레용 돈육      0.001370
어란(생선알)         0.001370
양념 돈육           0.001370
홍삼 간식           0.001370
파김치             0.001370
홍삼 음료           0.001370
물김치             0.001370
명태/동태           0.000000
닭가슴살            0.000000
반찬류             0.000000
더치커피            0.000000
배/포도/과일즙        0.000000
대게/킹크랩          0.000000
배추김치            0.

# 일단 일자별로 합치기..? 

## 가전

In [40]:
df_app_2018 = df_app_ts.loc[:, df_app_ts.columns.to_series().between('2018-01-01', '2018-12-31')]
df_app_2019 = df_app_ts.loc[:, df_app_ts.columns.to_series().between('2019-01-01', '2019-12-31')]

In [41]:
df_app_2018.shape, df_app_2019.shape

((40, 365), (40, 365))

In [45]:
df_app_2018.columns = pd.date_range('2019-01-01', '2019-12-31')

In [46]:
df_app_2018.head()

,2019-01-01,2019-01-02,2019-01-03,2019-01-04,2019-01-05,2019-01-06,2019-01-07,2019-01-08,2019-01-09,2019-01-10,2019-01-11,2019-01-12,2019-01-13,2019-01-14,2019-01-15,2019-01-16,2019-01-17,2019-01-18,2019-01-19,2019-01-20,2019-01-21,2019-01-22,2019-01-23,2019-01-24,2019-01-25,2019-01-26,2019-01-27,2019-01-28,2019-01-29,2019-01-30,2019-01-31,2019-02-01,2019-02-02,2019-02-03,2019-02-04,2019-02-05,2019-02-06,2019-02-07,2019-02-08,2019-02-09,2019-02-10,2019-02-11,2019-02-12,2019-02-13,2019-02-14,2019-02-15,2019-02-16,2019-02-17,2019-02-18,2019-02-19,2019-02-20,2019-02-21,2019-02-22,2019-02-23,2019-02-24,2019-02-25,2019-02-26,2019-02-27,2019-02-28,2019-03-01,2019-03-02,2019-03-03,2019-03-04,2019-03-05,2019-03-06,2019-03-07,2019-03-08,2019-03-09,2019-03-10,2019-03-11,2019-03-12,2019-03-13,2019-03-14,2019-03-15,2019-03-16,2019-03-17,2019-03-18,2019-03-19,2019-03-20,2019-03-21,2019-03-22,2019-03-23,2019-03-24,2019-03-25,2019-03-26,2019-03-27,2019-03-28,2019-03-29,2019-03-30,2019-03-31,2019-04-01,2019-04-02,2019-04-03,2019-04-04,2019-04-05,2019-04-06,2019-04-07,2019-04-08,2019-04-09,2019-04-10,2019-04-11,2019-04-12,2019-04-13,2019-04-14,2019-04-15,2019-04-16,2019-04-17,2019-04-18,2019-04-19,2019-04-20,2019-04-21,2019-04-22,2019-04-23,2019-04-24,2019-04-25,2019-04-26,2019-04-27,2019-04-28,2019-04-29,2019-04-30,2019-05-01,2019-05-02,2019-05-03,2019-05-04,2019-05-05,2019-05-06,2019-05-07,2019-05-08,2019-05-09,2019-05-10,2019-05-11,2019-05-12,2019-05-13,2019-05-14,2019-05-15,2019-05-16,2019-05-17,2019-05-18,2019-05-19,2019-05-20,2019-05-21,2019-05-22,2019-05-23,2019-05-24,2019-05-25,2019-05-26,2019-05-27,2019-05-28,2019-05-29,2019-05-30,2019-05-31,2019-06-01,2019-06-02,2019-06-03,2019-06-04,2019-06-05,2019-06-06,2019-06-07,2019-06-08,2019-06-09,2019-06-10,2019-06-11,2019-06-12,2019-06-13,2019-06-14,2019-06-15,2019-06-16,2019-06-17,2019-06-18,2019-06-19,2019-06-20,2019-06-21,2019-06-22,2019-06-23,2019-06-24,2019-06-25,2019-06-26,2019-06-27,2019-06-28,2019-06-29,2019-06-30,2019-07-01,2019-07-02,2019-07-03,2019-07-04,2019-07-05,2019-07-06,2019-07-07,2019-07-08,2019-07-09,2019-07-10,2019-07-11,2019-07-12,2019-07-13,2019-07-14,2019-07-15,2019-07-16,2019-07-17,2019-07-18,2019-07-19,2019-07-20,2019-07-21,2019-07-22,2019-07-23,2019-07-24,2019-07-25,2019-07-26,2019-07-27,2019-07-28,2019-07-29,2019-07-30,2019-07-31,2019-08-01,2019-08-02,2019-08-03,2019-08-04,2019-08-05,2019-08-06,2019-08-07,2019-08-08,2019-08-09,2019-08-10,2019-08-11,2019-08-12,2019-08-13,2019-08-14,2019-08-15,2019-08-16,2019-08-17,2019-08-18,2019-08-19,2019-08-20,2019-08-21,2019-08-22,2019-08-23,2019-08-24,2019-08-25,2019-08-26,2019-08-27,2019-08-28,2019-08-29,2019-08-30,2019-08-31,2019-09-01,2019-09-02,2019-09-03,2019-09-04,2019-09-05,2019-09-06,2019-09-07,2019-09-08,2019-09-09,2019-09-10,2019-09-11,2019-09-12,2019-09-13,2019-09-14,2019-09-15,2019-09-16,2019-09-17,2019-09-18,2019-09-19,2019-09-20,2019-09-21,2019-09-22,2019-09-23,2019-09-24,2019-09-25,2019-09-26,2019-09-27,2019-09-28,2019-09-29,2019-09-30,2019-10-01,2019-10-02,2019-10-03,2019-10-04,2019-10-05,2019-10-06,2019-10-07,2019-10-08,2019-10-09,2019-10-10,2019-10-11,2019-10-12,2019-10-13,2019-10-14,2019-10-15,2019-10-16,2019-10-17,2019-10-18,2019-10-19,2019-10-20,2019-10-21,2019-10-22,2019-10-23,2019-10-24,2019-10-25,2019-10-26,2019-10-27,2019-10-28,2019-10-29,2019-10-30,2019-10-31,2019-11-01,2019-11-02,2019-11-03,2019-11-04,2019-11-05,2019-11-06,2019-11-07,2019-11-08,2019-11-09,2019-11-10,2019-11-11,2019-11-12,2019-11-13,2019-11-14,2019-11-15,2019-11-16,2019-11-17,2019-11-18,2019-11-19,2019-11-20,2019-11-21,2019-11-22,2019-11-23,2019-11-24,2019-11-25,2019-11-26,2019-11-27,2019-11-28,2019-11-29,2019-11-30,2019-12-01,2019-12-02,2019-12-03,2019-12-04,2019-12-05,2019-12-06,2019-12-07,2019-12-08,2019-12-09,2019-12-10,2019-12-11,2019-12-12,2019-12-13,2019-12-14,2019-12-15,2019-12-16,2019-12-17,2019-12-18,2019-12-19,2019-12-20,2019-12-21,2019-12-22,2019-12-23,2019-12-24,2019-12-25,2019-12-26,2019-12-27,2019-12-28,2019-12-29,2019-1

In [48]:
df_app_2018 = df_app_2018.fillna(0)
df_app_2019 = df_app_2019.fillna(0)

In [49]:
df_app_ts_2 = df_app_2018 + df_app_2019
df_app_ts_2.head()

,2019-01-01,2019-01-02,2019-01-03,2019-01-04,2019-01-05,2019-01-06,2019-01-07,2019-01-08,2019-01-09,2019-01-10,2019-01-11,2019-01-12,2019-01-13,2019-01-14,2019-01-15,2019-01-16,2019-01-17,2019-01-18,2019-01-19,2019-01-20,2019-01-21,2019-01-22,2019-01-23,2019-01-24,2019-01-25,2019-01-26,2019-01-27,2019-01-28,2019-01-29,2019-01-30,2019-01-31,2019-02-01,2019-02-02,2019-02-03,2019-02-04,2019-02-05,2019-02-06,2019-02-07,2019-02-08,2019-02-09,2019-02-10,2019-02-11,2019-02-12,2019-02-13,2019-02-14,2019-02-15,2019-02-16,2019-02-17,2019-02-18,2019-02-19,2019-02-20,2019-02-21,2019-02-22,2019-02-23,2019-02-24,2019-02-25,2019-02-26,2019-02-27,2019-02-28,2019-03-01,2019-03-02,2019-03-03,2019-03-04,2019-03-05,2019-03-06,2019-03-07,2019-03-08,2019-03-09,2019-03-10,2019-03-11,2019-03-12,2019-03-13,2019-03-14,2019-03-15,2019-03-16,2019-03-17,2019-03-18,2019-03-19,2019-03-20,2019-03-21,2019-03-22,2019-03-23,2019-03-24,2019-03-25,2019-03-26,2019-03-27,2019-03-28,2019-03-29,2019-03-30,2019-03-31,2019-04-01,2019-04-02,2019-04-03,2019-04-04,2019-04-05,2019-04-06,2019-04-07,2019-04-08,2019-04-09,2019-04-10,2019-04-11,2019-04-12,2019-04-13,2019-04-14,2019-04-15,2019-04-16,2019-04-17,2019-04-18,2019-04-19,2019-04-20,2019-04-21,2019-04-22,2019-04-23,2019-04-24,2019-04-25,2019-04-26,2019-04-27,2019-04-28,2019-04-29,2019-04-30,2019-05-01,2019-05-02,2019-05-03,2019-05-04,2019-05-05,2019-05-06,2019-05-07,2019-05-08,2019-05-09,2019-05-10,2019-05-11,2019-05-12,2019-05-13,2019-05-14,2019-05-15,2019-05-16,2019-05-17,2019-05-18,2019-05-19,2019-05-20,2019-05-21,2019-05-22,2019-05-23,2019-05-24,2019-05-25,2019-05-26,2019-05-27,2019-05-28,2019-05-29,2019-05-30,2019-05-31,2019-06-01,2019-06-02,2019-06-03,2019-06-04,2019-06-05,2019-06-06,2019-06-07,2019-06-08,2019-06-09,2019-06-10,2019-06-11,2019-06-12,2019-06-13,2019-06-14,2019-06-15,2019-06-16,2019-06-17,2019-06-18,2019-06-19,2019-06-20,2019-06-21,2019-06-22,2019-06-23,2019-06-24,2019-06-25,2019-06-26,2019-06-27,2019-06-28,2019-06-29,2019-06-30,2019-07-01,2019-07-02,2019-07-03,2019-07-04,2019-07-05,2019-07-06,2019-07-07,2019-07-08,2019-07-09,2019-07-10,2019-07-11,2019-07-12,2019-07-13,2019-07-14,2019-07-15,2019-07-16,2019-07-17,2019-07-18,2019-07-19,2019-07-20,2019-07-21,2019-07-22,2019-07-23,2019-07-24,2019-07-25,2019-07-26,2019-07-27,2019-07-28,2019-07-29,2019-07-30,2019-07-31,2019-08-01,2019-08-02,2019-08-03,2019-08-04,2019-08-05,2019-08-06,2019-08-07,2019-08-08,2019-08-09,2019-08-10,2019-08-11,2019-08-12,2019-08-13,2019-08-14,2019-08-15,2019-08-16,2019-08-17,2019-08-18,2019-08-19,2019-08-20,2019-08-21,2019-08-22,2019-08-23,2019-08-24,2019-08-25,2019-08-26,2019-08-27,2019-08-28,2019-08-29,2019-08-30,2019-08-31,2019-09-01,2019-09-02,2019-09-03,2019-09-04,2019-09-05,2019-09-06,2019-09-07,2019-09-08,2019-09-09,2019-09-10,2019-09-11,2019-09-12,2019-09-13,2019-09-14,2019-09-15,2019-09-16,2019-09-17,2019-09-18,2019-09-19,2019-09-20,2019-09-21,2019-09-22,2019-09-23,2019-09-24,2019-09-25,2019-09-26,2019-09-27,2019-09-28,2019-09-29,2019-09-30,2019-10-01,2019-10-02,2019-10-03,2019-10-04,2019-10-05,2019-10-06,2019-10-07,2019-10-08,2019-10-09,2019-10-10,2019-10-11,2019-10-12,2019-10-13,2019-10-14,2019-10-15,2019-10-16,2019-10-17,2019-10-18,2019-10-19,2019-10-20,2019-10-21,2019-10-22,2019-10-23,2019-10-24,2019-10-25,2019-10-26,2019-10-27,2019-10-28,2019-10-29,2019-10-30,2019-10-31,2019-11-01,2019-11-02,2019-11-03,2019-11-04,2019-11-05,2019-11-06,2019-11-07,2019-11-08,2019-11-09,2019-11-10,2019-11-11,2019-11-12,2019-11-13,2019-11-14,2019-11-15,2019-11-16,2019-11-17,2019-11-18,2019-11-19,2019-11-20,2019-11-21,2019-11-22,2019-11-23,2019-11-24,2019-11-25,2019-11-26,2019-11-27,2019-11-28,2019-11-29,2019-11-30,2019-12-01,2019-12-02,2019-12-03,2019-12-04,2019-12-05,2019-12-06,2019-12-07,2019-12-08,2019-12-09,2019-12-10,2019-12-11,2019-12-12,2019-12-13,2019-12-14,2019-12-15,2019-12-16,2019-12-17,2019-12-18,2019-12-19,2019-12-20,2019-12-21,2019-12-22,2019-12-23,2019-12-24,2019-12-25,2019-12-26,2019-12-27,2019-12-28,2019-12-29,2019-1

In [50]:
df_app_ts_2.shape

(40, 365)

In [52]:
model1 = TimeSeriesKMeans(n_clusters=5, metric="dtw", max_iter=5, n_jobs=-1)

for i in tqdm(range(len(df_app_ts_2))) : 
    model1.fit(df_app_ts_2)

100%|██████████| 40/40 [05:57<00:00,  8.93s/it]


In [55]:
model1.labels_

array([1, 1, 2, 2, 1, 3, 1, 1, 1, 3, 1, 1, 1, 4, 1, 1, 3, 1, 1, 1, 1, 1,
       4, 4, 4, 4, 1, 1, 1, 4, 3, 1, 1, 0, 1, 1, 1, 1, 3, 0])

In [56]:
df_app_ts_2.index

Index(['가스온수기', '가열식 가습기', '공기정화 용품', '공기청정기', '난방용 열풍기', '냉풍기', '돈풍기',
       '라디에이터', '멀티형 에어컨', '벽걸이 에어컨', '벽걸이형 냉온풍기', '벽걸이형 선풍기', '보일러',
       '복합식 가습기', '산림욕기', '스탠드형 냉온풍기', '스탠드형 에어컨', '신발건조기', '업소용 선풍기', '에어워셔',
       '에어커튼', '에어컨 리모컨', '온수매트', '온열매트', '온풍기', '의류건조기', '이동형 에어컨', '자연식 가습기',
       '전기온수기', '전기장판', '제습기', '중대형 에어컨', '천장형 에어컨', '초음파식 가습기', '카페트매트',
       '컨벡터', '탁상/USB 선풍기', '황토매트', '휴대용 선풍기', '히터'],
      dtype='object', name='sm_cat')

In [57]:
df_app_result = pd.DataFrame({
    'sm_cat' : df_app_ts_2.index, 
    'cluster' : model1.labels_
})

df_app_result

,sm_cat,cluster
0,가스온수기,1
1,가열식 가습기,1
2,공기정화 용품,2
3,공기청정기,2
4,난방용 열풍기,1
5,냉풍기,3
6,돈풍기,1
7,라디에이터,1
8,멀티형 에어컨,1
9,벽걸이 에어컨,3


In [58]:
# cluster 0
df_app_result[df_app_result.cluster==0]

,sm_cat,cluster
33,초음파식 가습기,0
39,히터,0


In [59]:
# cluster 1
df_app_result[df_app_result.cluster==1]

,sm_cat,cluster
0,가스온수기,1
1,가열식 가습기,1
4,난방용 열풍기,1
6,돈풍기,1
7,라디에이터,1
8,멀티형 에어컨,1
10,벽걸이형 냉온풍기,1
11,벽걸이형 선풍기,1
12,보일러,1
14,산림욕기,1


In [60]:
# cluster 2
df_app_result[df_app_result.cluster==2]

,sm_cat,cluster
2,공기정화 용품,2
3,공기청정기,2


In [61]:
# cluster 3
df_app_result[df_app_result.cluster==3]

,sm_cat,cluster
5,냉풍기,3
9,벽걸이 에어컨,3
16,스탠드형 에어컨,3
30,제습기,3
38,휴대용 선풍기,3


In [62]:
# cluster 4
df_app_result[df_app_result.cluster==4]

,sm_cat,cluster
13,복합식 가습기,4
22,온수매트,4
23,온열매트,4
24,온풍기,4
25,의류건조기,4
29,전기장판,4


## 뷰티 

In [63]:
df_bt_2018 = df_bt_ts.loc[:, df_bt_ts.columns.to_series().between('2018-01-01', '2018-12-31')]
df_bt_2019 = df_bt_ts.loc[:, df_bt_ts.columns.to_series().between('2019-01-01', '2019-12-31')]

df_bt_2018.columns = pd.date_range('2019-01-01', '2019-12-31')

df_bt_2018 = df_bt_2018.fillna(0)
df_bt_2019 = df_bt_2019.fillna(0)

df_bt_ts_2 = df_bt_2018 + df_bt_2019
print(df_bt_ts_2.shape)
df_bt_ts_2.head()

(131, 365)


,2019-01-01,2019-01-02,2019-01-03,2019-01-04,2019-01-05,2019-01-06,2019-01-07,2019-01-08,2019-01-09,2019-01-10,2019-01-11,2019-01-12,2019-01-13,2019-01-14,2019-01-15,2019-01-16,2019-01-17,2019-01-18,2019-01-19,2019-01-20,2019-01-21,2019-01-22,2019-01-23,2019-01-24,2019-01-25,2019-01-26,2019-01-27,2019-01-28,2019-01-29,2019-01-30,2019-01-31,2019-02-01,2019-02-02,2019-02-03,2019-02-04,2019-02-05,2019-02-06,2019-02-07,2019-02-08,2019-02-09,2019-02-10,2019-02-11,2019-02-12,2019-02-13,2019-02-14,2019-02-15,2019-02-16,2019-02-17,2019-02-18,2019-02-19,2019-02-20,2019-02-21,2019-02-22,2019-02-23,2019-02-24,2019-02-25,2019-02-26,2019-02-27,2019-02-28,2019-03-01,2019-03-02,2019-03-03,2019-03-04,2019-03-05,2019-03-06,2019-03-07,2019-03-08,2019-03-09,2019-03-10,2019-03-11,2019-03-12,2019-03-13,2019-03-14,2019-03-15,2019-03-16,2019-03-17,2019-03-18,2019-03-19,2019-03-20,2019-03-21,2019-03-22,2019-03-23,2019-03-24,2019-03-25,2019-03-26,2019-03-27,2019-03-28,2019-03-29,2019-03-30,2019-03-31,2019-04-01,2019-04-02,2019-04-03,2019-04-04,2019-04-05,2019-04-06,2019-04-07,2019-04-08,2019-04-09,2019-04-10,2019-04-11,2019-04-12,2019-04-13,2019-04-14,2019-04-15,2019-04-16,2019-04-17,2019-04-18,2019-04-19,2019-04-20,2019-04-21,2019-04-22,2019-04-23,2019-04-24,2019-04-25,2019-04-26,2019-04-27,2019-04-28,2019-04-29,2019-04-30,2019-05-01,2019-05-02,2019-05-03,2019-05-04,2019-05-05,2019-05-06,2019-05-07,2019-05-08,2019-05-09,2019-05-10,2019-05-11,2019-05-12,2019-05-13,2019-05-14,2019-05-15,2019-05-16,2019-05-17,2019-05-18,2019-05-19,2019-05-20,2019-05-21,2019-05-22,2019-05-23,2019-05-24,2019-05-25,2019-05-26,2019-05-27,2019-05-28,2019-05-29,2019-05-30,2019-05-31,2019-06-01,2019-06-02,2019-06-03,2019-06-04,2019-06-05,2019-06-06,2019-06-07,2019-06-08,2019-06-09,2019-06-10,2019-06-11,2019-06-12,2019-06-13,2019-06-14,2019-06-15,2019-06-16,2019-06-17,2019-06-18,2019-06-19,2019-06-20,2019-06-21,2019-06-22,2019-06-23,2019-06-24,2019-06-25,2019-06-26,2019-06-27,2019-06-28,2019-06-29,2019-06-30,2019-07-01,2019-07-02,2019-07-03,2019-07-04,2019-07-05,2019-07-06,2019-07-07,2019-07-08,2019-07-09,2019-07-10,2019-07-11,2019-07-12,2019-07-13,2019-07-14,2019-07-15,2019-07-16,2019-07-17,2019-07-18,2019-07-19,2019-07-20,2019-07-21,2019-07-22,2019-07-23,2019-07-24,2019-07-25,2019-07-26,2019-07-27,2019-07-28,2019-07-29,2019-07-30,2019-07-31,2019-08-01,2019-08-02,2019-08-03,2019-08-04,2019-08-05,2019-08-06,2019-08-07,2019-08-08,2019-08-09,2019-08-10,2019-08-11,2019-08-12,2019-08-13,2019-08-14,2019-08-15,2019-08-16,2019-08-17,2019-08-18,2019-08-19,2019-08-20,2019-08-21,2019-08-22,2019-08-23,2019-08-24,2019-08-25,2019-08-26,2019-08-27,2019-08-28,2019-08-29,2019-08-30,2019-08-31,2019-09-01,2019-09-02,2019-09-03,2019-09-04,2019-09-05,2019-09-06,2019-09-07,2019-09-08,2019-09-09,2019-09-10,2019-09-11,2019-09-12,2019-09-13,2019-09-14,2019-09-15,2019-09-16,2019-09-17,2019-09-18,2019-09-19,2019-09-20,2019-09-21,2019-09-22,2019-09-23,2019-09-24,2019-09-25,2019-09-26,2019-09-27,2019-09-28,2019-09-29,2019-09-30,2019-10-01,2019-10-02,2019-10-03,2019-10-04,2019-10-05,2019-10-06,2019-10-07,2019-10-08,2019-10-09,2019-10-10,2019-10-11,2019-10-12,2019-10-13,2019-10-14,2019-10-15,2019-10-16,2019-10-17,2019-10-18,2019-10-19,2019-10-20,2019-10-21,2019-10-22,2019-10-23,2019-10-24,2019-10-25,2019-10-26,2019-10-27,2019-10-28,2019-10-29,2019-10-30,2019-10-31,2019-11-01,2019-11-02,2019-11-03,2019-11-04,2019-11-05,2019-11-06,2019-11-07,2019-11-08,2019-11-09,2019-11-10,2019-11-11,2019-11-12,2019-11-13,2019-11-14,2019-11-15,2019-11-16,2019-11-17,2019-11-18,2019-11-19,2019-11-20,2019-11-21,2019-11-22,2019-11-23,2019-11-24,2019-11-25,2019-11-26,2019-11-27,2019-11-28,2019-11-29,2019-11-30,2019-12-01,2019-12-02,2019-12-03,2019-12-04,2019-12-05,2019-12-06,2019-12-07,2019-12-08,2019-12-09,2019-12-10,2019-12-11,2019-12-12,2019-12-13,2019-12-14,2019-12-15,2019-12-16,2019-12-17,2019-12-18,2019-12-19,2019-12-20,2019-12-21,2019-12-22,2019-12-23,2019-12-24,2019-12-25,2019-12-26,2019-12-27,2019-12-28,2019-12-29,2019-1

In [66]:
model2 = TimeSeriesKMeans(n_clusters=10, metric="dtw", max_iter=5, n_jobs=-1)

for i in tqdm(range(len(df_bt_ts_2))) : 
    model2.fit(df_bt_ts_2)

100%|██████████| 131/131 [1:38:18<00:00, 45.03s/it]


In [67]:
df_beauty_result = pd.DataFrame({
    'sm_cat' : df_bt_ts_2.index, 
    'cluster' : model2.labels_
})

df_beauty_result

,sm_cat,cluster
0,기능성 링클케어 화장품,0
1,기능성 모공관리 화장품,0
2,기능성 아이케어 화장품,8
3,기능성 영양보습 화장품,0
4,기능성 트러블케어 화장품,0
5,기능성 화이트닝 화장품,0
6,기능성 화장품 세트,0
7,기초 화장용 로션,9
8,기초 화장용 미스트,7
9,기초 화장용 스킨,9


In [68]:
df_beauty_result[df_beauty_result.cluster==0]

,sm_cat,cluster
0,기능성 링클케어 화장품,0
1,기능성 모공관리 화장품,0
3,기능성 영양보습 화장품,0
4,기능성 트러블케어 화장품,0
5,기능성 화이트닝 화장품,0
6,기능성 화장품 세트,0
13,남성 로션,0
14,남성 메이크업,0
15,남성 선케어,0
17,남성 쉐이빙,0


In [69]:
df_beauty_result[df_beauty_result.cluster==1]

,sm_cat,cluster
23,네일 메이크업 용품,1
93,스킨케어 마스크팩,1


In [70]:
df_beauty_result[df_beauty_result.cluster==2]

,sm_cat,cluster
12,기초 화장용 크림,2


In [71]:
df_beauty_result[df_beauty_result.cluster==3]

,sm_cat,cluster
11,기초 화장용 오일/앰플,3
39,바디 클렌져,3
54,베이스 메이크업용 파우더팩트,3
55,베이스 메이크업용 파운데이션,3
87,선크림,3
121,헤어스타일링용 염색약,3


In [72]:
df_beauty_result[df_beauty_result.cluster==4]

,sm_cat,cluster
83,샴푸,4


In [73]:
df_beauty_result[df_beauty_result.cluster==5]

,sm_cat,cluster
16,남성 세트,5
19,남성 에센스,5
31,린스,5
38,바디 스크럽,5
45,바디케어용 청 결제,5
47,베이스 메이크업용 BB크림,5
50,베이스 메이크업용 메이크업베이스,5
51,베이스 메이크업용 컨실러,5
56,베이스 메이크업용 프라이머,5
62,뷰티 화장솜,5


In [74]:
df_beauty_result[df_beauty_result.cluster==6]

,sm_cat,cluster
10,기초 화장용 에센스,6


In [75]:
df_beauty_result[df_beauty_result.cluster==7]

,sm_cat,cluster
8,기초 화장용 미스트,7
34,메이크업 브러쉬,7
71,색조 메이크업 립밤,7
74,색조 메이크업 마스카라,7
79,색조 메이크업 아이라이너,7
80,색조 메이크업 아이브로우,7
81,색조 메이크업 아이섀도우,7
91,스크럽/필링크림,7
95,스킨케어 시트마스크팩,7
103,클렌징 오일,7


In [76]:
df_beauty_result[df_beauty_result.cluster==8]

,sm_cat,cluster
2,기능성 아이케어 화장품,8
52,베이스 메이크업용 쿠션팩트,8
109,트리트먼트,8


In [77]:
df_beauty_result[df_beauty_result.cluster==9]

,sm_cat,cluster
7,기초 화장용 로션,9
9,기초 화장용 스킨,9
36,바디 보습제,9
72,색조 메이크업 립스틱,9
73,색조 메이크업 립틴트,9
107,클렌징 폼,9
125,헤어에센스,9
